# Agents

The core idea of agents is to use an LLM to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order. You can learn more about agents [here](https://python.langchain.com/docs/modules/agents/). In this notebook, you'll learn how to create an agent that can interact with Wikipedia. 

## 1. Import packages

In [1]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import Tool

## 2. Setup the language model

So far, you learned how to construct a language model without any parameters. But in order to improve the performance of the language model, you can specify some parameters. 

Here, I changed the value for the `temperature`. The higher the temperature, the more random the output. The lower the temperature, the more predictable the output.

I also specified the `model` to be used. In this case, I'm using the `gpt-3.5-turbo-16k` to accomodate larger conversation history. If your API key is not authorized to use this model, you can use the `gpt-3.5-turbo` model instead.

In [2]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")

## 3. Setup the conversational memory

Our agent will use `ConversationBufferMemory` for its memory. At the core of this object is a `ChatMessageHistory` object that wraps the list of `HumanMessage` and `AIMessage` in our conversation.

We also added an initial `AIMessage` into the `ChatMessageHistory` object. This is to prime the language model with some context.

In [3]:
messages = ChatMessageHistory()
memory = ConversationBufferMemory(
    chat_memory=messages,
    return_messages=True,
    memory_key="chat_history",
    output_key="output",
)

# Add the initial instruction for the language model
messages.add_ai_message(
    "As a helpful Wikipedia assistant, I can answer your questions accurately. "
    "Try asking me a question about a topic you're interested in."
)

memory.load_memory_variables({})

{'chat_history': [AIMessage(content="As a helpful Wikipedia assistant, I can answer your questions accurately. Try asking me a question about a topic you're interested in.")]}

## 4. Setup the tools

Tools are functions that an agent can use during its reasoning process. In this case, we're using the `wikipedia` tool to query Wikipedia. We can even create our own tools and add them to the agent. For example, we can create a tool that can query the weather, or perform some calculations accurately. You can learn more about tools [here](https://python.langchain.com/docs/modules/agents/tools/).

In [4]:
wikipedia = WikipediaAPIWrapper()

tools = [
    Tool(
        name="Wikipedia Search",
        func=wikipedia.run,
        description="Useful for when you need to look up a definition or a short description of something. Be specific with your input.",
    ),
]

## 5. Setup the agent

Finally, we instantiate the agent with the language model, memory and tools we created. We also need to specify the type of the agent. In this case, we're using the `CHAT_CONVERSATIONAL_REACT_DESCRIPTION` type. You can learn more about agent types [here](https://python.langchain.com/docs/modules/agents/agent_types/).

In [5]:
agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION
agent_executor = initialize_agent(
    llm=llm,
    tools=tools,
    agent=agent,
    memory=memory,
    verbose=True,
)

## 6. Start the conversation

In [6]:
response = agent_executor("Hello!")
print(response["output"])



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Hello! How can I assist you today?"
}

> Finished chain.
Hello! How can I assist you today?


Let's try to ask the agent a question whose answer can be found in Wikipedia. 

In [7]:
response = agent_executor("What is a Fakemon?")
print(response["output"])



> Entering new AgentExecutor chain...
{
    "action": "Wikipedia Search",
    "action_input": "Fakemon"
}
Observation: Page: Fakemon
Summary: Fakemon, also called Fakémon, are fan-designed fictional creatures based on the Pokémon franchise of monster-taming games. While many such designs have been created purely as fan art, others were made specifically as hoaxes to fool fans into believing they will appear in future series titles.

Page: Main Page
Summary: 

Page: List of generation I Pokémon
Summary: The first generation (Generation I) of the Pokémon franchise features the original 151 fictional species of monsters introduced to the core video game series in the 1996 Game Boy games Pocket Monsters Red and Green (known as Pokémon Red and Blue outside of Japan).
The following list details the 151 Pokémon of generation I in order of their National Pokédex number. The first Pokémon, Bulbasaur, is number 001 and the last, Mew, is number 151. Alternate forms that result in type changes a

Notice that the agent is able to answer the question correctly. It's also able to search for the answer in Wikipedia.